In [1]:
import os
import os.path as op
import numpy as np
import nibabel as nib
import pandas as pd
from nilearn.maskers import NiftiMasker
from nilearn import datasets
from nilearn.image import threshold_img

## Define Dice Coefficient Function
The Dice coefficient measures the overlap between two binary masks, ranging from 0 (no overlap) to 1 (perfect overlap).

In [8]:
def dice_coefficient(mask1, mask2, brain_mask=None):
    """
    Calculate Dice similarity coefficient between two binary masks.
    
    Dice = 2 * |X ∩ Y| / (|X| + |Y|)
    
    Parameters:
    -----------
    mask1, mask2 : array-like
        Binary masks (1D or 3D arrays)
    brain_mask : array-like, optional
        Binary mask to restrict analysis to brain voxels
    
    Returns:
    --------
    dice : float
        Dice coefficient (0-1)
    """
    # Flatten arrays if needed
    mask1_flat = mask1.flatten()
    mask2_flat = mask2.flatten()
    
    # Apply brain mask if provided
    if brain_mask is not None:
        brain_mask_flat = brain_mask.flatten().astype(bool)
        mask1_flat = mask1_flat[brain_mask_flat]
        mask2_flat = mask2_flat[brain_mask_flat]
    
    # Calculate intersection and sizes
    intersection = np.sum(mask1_flat * mask2_flat)
    size1 = np.sum(mask1_flat)
    size2 = np.sum(mask2_flat)
    
    # Avoid division by zero
    if size1 + size2 == 0:
        return 0.0
    
    dice = (2.0 * intersection) / (size1 + size2)
    return dice

## Setup Directories and Masker

In [2]:
data_dir = "./dset"
analysis_dir = op.join(data_dir, "hb-conn")
output_dir = "./derivatives"
os.makedirs(output_dir, exist_ok=True)

# Setup masker
mask_img = datasets.load_mni152_brain_mask(resolution=1)
masker = NiftiMasker(mask_img=mask_img)
masker = masker.fit()

# Load MNI152 template to use as brain mask
template_img = datasets.load_mni152_template(resolution=1)
template_data = template_img.get_fdata()
brain_mask = (template_data != 0).astype(int)

print(f"Brain mask shape: {brain_mask.shape}")
print(f"Brain voxels: {np.sum(brain_mask)}")

Brain mask shape: (197, 233, 189)
Brain voxels: 1886539


## Define File Paths

In [6]:
# Group directories
group_drawn_dir = op.join(analysis_dir, "group-subj/habenula")
group_atlas_dir = op.join(analysis_dir, "group-atlas/habenula")

# File paths for group average (1-sample) maps
drawn_1s_fn = op.join(group_drawn_dir, "averaged", "sub-group_task-rest_desc-1SampletTest_thresh.nii.gz")
atlas_1s_fn = op.join(group_atlas_dir, "averaged", "sub-group_task-rest_desc-1SampletTest_thresh.nii.gz")
# File paths for group comparison (2-sample) maps
drawn_2s_fn = op.join(group_drawn_dir, "difference", "sub-group_task-rest_desc-2SampletTest_thresh.nii.gz")
atlas_2s_fn = op.join(group_atlas_dir, "difference", "sub-group_task-rest_desc-2SampletTest_thresh.nii.gz")

print("File paths defined:")
print(f"Drawn 1-sample: {drawn_1s_fn}")
print(f"Avg 1-sample: {atlas_1s_fn}")
print(f"Drawn 2-sample: {drawn_2s_fn}")
print(f"Avg 2-sample: {atlas_2s_fn}")

File paths defined:
Drawn 1-sample: ./dset/hb-conn/group-subj/habenula/averaged/sub-group_task-rest_desc-1SampletTest_thresh.nii.gz
Avg 1-sample: ./dset/hb-conn/group-atlas/habenula/averaged/sub-group_task-rest_desc-1SampletTest_thresh.nii.gz
Drawn 2-sample: ./dset/hb-conn/group-subj/habenula/difference/sub-group_task-rest_desc-2SampletTest_thresh.nii.gz
Avg 2-sample: ./dset/hb-conn/group-atlas/habenula/difference/sub-group_task-rest_desc-2SampletTest_thresh.nii.gz


## Calculate Dice Coefficients for Group Average Maps (1-Sample)

In [9]:
drawn_1s_img = nib.load(drawn_1s_fn)
atlas_1s_img = nib.load(atlas_1s_fn)
    
# Convert to arrays (masker already applies brain mask)
drawn_1s_arr = masker.transform(drawn_1s_img)
atlas_1s_arr = masker.transform(atlas_1s_img)
    
# Create binary masks (any non-zero value = 1)
drawn_1s_binary = (drawn_1s_arr != 0).astype(int)
atlas_1s_binary = (atlas_1s_arr != 0).astype(int)
    
# Calculate binary Dice coefficient (masker already restricts to brain voxels)
dice_1s = dice_coefficient(drawn_1s_binary, atlas_1s_binary)
 
print(f"1-sample Dice coefficient: {dice_1s:.4f}")

/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


1-sample Dice coefficient: 1.0000


## Calculate Dice Coefficients for Group Difference Maps (2-Sample)

### (ASD>NT)

In [10]:
drawn_2s_img = nib.load(drawn_2s_fn)
atlas_2s_img = nib.load(atlas_2s_fn)
    
# Convert to arrays (masker already applies brain mask)
drawn_2s_arr = masker.transform(drawn_2s_img)
atlas_2s_arr = masker.transform(atlas_2s_img)
    
# Create binary masks (any non-zero value = 1)
drawn_2s_binary = (drawn_2s_arr != 0).astype(int)
atlas_2s_binary = (atlas_2s_arr != 0).astype(int)
    
# Calculate binary Dice coefficient (masker already restricts to brain voxels)
dice_2s = dice_coefficient(drawn_2s_binary, atlas_2s_binary)
 
print(f"2-sample Dice coefficient: {dice_2s:.4f}")

/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


2-sample Dice coefficient: 0.5474


## Define Spearman Correlation Function
The Spearman correlation measures the monotonic relationship between two continuous (thresholded) masks.

In [17]:
from scipy.stats import spearmanr

def spearman_correlation(mask1, mask2, brain_mask=None, method='overlap'):
    """
    Calculate Spearman correlation between two thresholded masks.
    
    Parameters:
    -----------
    mask1, mask2 : array-like
        Continuous masks (1D or 3D arrays) - typically thresholded statistical maps
    brain_mask : array-like, optional
        Binary mask to restrict analysis to brain voxels
    method : str, default='overlap'
        'overlap': correlate only where both masks are non-zero (RECOMMENDED)
        'union': correlate where either mask is non-zero
        'all': correlate all brain voxels (if brain_mask provided)
    
    Returns:
    --------
    correlation : float
        Spearman correlation coefficient (-1 to 1)
    p_value : float
        Two-tailed p-value for the correlation
    """
    # Flatten arrays if needed
    mask1_flat = mask1.flatten()
    mask2_flat = mask2.flatten()
    
    # Apply brain mask if provided
    if brain_mask is not None:
        brain_mask_flat = brain_mask.flatten().astype(bool)
        mask1_flat = mask1_flat[brain_mask_flat]
        mask2_flat = mask2_flat[brain_mask_flat]
    
    # Select voxels based on method
    if method == 'overlap':
        # Only correlate where BOTH masks are non-zero
        valid_mask = (mask1_flat != 0) & (mask2_flat != 0)
    elif method == 'union':
        # Correlate where EITHER mask is non-zero
        valid_mask = (mask1_flat != 0) | (mask2_flat != 0)
    elif method == 'all':
        # Use all voxels (requires brain_mask to be meaningful)
        valid_mask = np.ones_like(mask1_flat, dtype=bool)
    else:
        raise ValueError("method must be 'overlap', 'union', or 'all'")
    
    mask1_valid = mask1_flat[valid_mask]
    mask2_valid = mask2_flat[valid_mask]
    
    # Check if we have enough data points
    if len(mask1_valid) < 2:
        return np.nan, np.nan
    
    # Calculate Spearman correlation
    correlation, p_value = spearmanr(mask1_valid, mask2_valid)
    
    return correlation, p_value

In [21]:
drawn_1s_img = nib.load(drawn_1s_fn)
atlas_1s_img = nib.load(atlas_1s_fn)
    
# Convert to arrays (masker already applies brain mask)
drawn_1s_arr = masker.transform(drawn_1s_img)
atlas_1s_arr = masker.transform(atlas_1s_img)
    
# Calculate Spearman correlation (masker already restricts to brain voxels)
correlation_1s, p_value_1s = spearman_correlation(drawn_1s_arr, atlas_1s_arr)
 
print(f"1-sample Spearman Correlation: {correlation_1s:.4f}")
print(f"1-sample p-value: {p_value_1s:.12f}")

/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


1-sample Spearman Correlation: 0.6122
1-sample p-value: 0.000000000000


In [22]:
drawn_2s_img = nib.load(drawn_2s_fn)
atlas_2s_img = nib.load(atlas_2s_fn)
    
# Convert to arrays (masker already applies brain mask)
drawn_2s_arr = masker.transform(drawn_2s_img)
atlas_2s_arr = masker.transform(atlas_2s_img)
    
# Calculate Spearman correlation (masker already restricts to brain voxels)
correlation_2s, p_value_2s = spearman_correlation(drawn_2s_arr, atlas_2s_arr)
 
print(f"2-sample Spearman Correlation: {correlation_2s:.4f}")
print(f"2-sample p-value: {p_value_2s:.12f}")

/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/chloehampson/Desktop/habenula-comparison/.venv/lib/python3.9/site-packages/nilearn/maskers/nifti_masker.py:108: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


2-sample Spearman Correlation: 0.9394
2-sample p-value: 0.000000000000
